# Spatiocyte simulations with single-molecule resolution

We showed an example of E-Cell4 spatial representation.  
Next let's simulate the models with more detailed spatial representation called **single molecule resolution**.

## Spatiocyte lattice-based method

In spatical Gillespie method, we divided the **Space** into smaller **Space**, then we diffuse the molecules in the **Subvolume**.
However we simulated the molecules in the **Subvolume** as the number of the molecules, and the location of the molecules are NOT determinated.

In other words, the spatical resolution of spatical Gillespie method is equal to the side of the **Subvolume** $l$.
To improve this resolution we need to make the size of $l$ small, but in this method the $l$ must be larger than the (at least) 10 times the diameter of molecule $R$.

How can we improve the spatical resolution to the size of the molecule?
The answer is the simulation with single-molecule resolution.
This method simulate the molecule not with the number of the molecules, but with the spatical reaction diffusion of each molecule.

E-Cell4 has multiple single-molecule resolution method, here we explain about Spatiocyte lattice-based method.

Spatiocyte treat each molecule as hard sphere and diffuse the molecules in hexagonal close-packed lattice.

Spatiocyte has ID for each molecule and the position of the molecule with single-molecule resolution.
To use the time scale, Spatiocyte has 100 times smaller time-step than spatical Gillespie, because the time scale of diffusion increases with the square of the distance.

Next, let's use Spatiocyte.


In [1]:
from ecell4 import *

with species_attributes():
    A | B | C | {'D': '1'}

with reaction_rules():
    A + B == C | (0.01, 0.3)

m = get_model()
w = lattice.LatticeWorld(Real3(1, 1, 1), 0.005)  # The second argument is 'voxel_radius'.
w.bind_to(m)
w.add_molecules(Species('C'), 60)
sim = lattice.LatticeSimulator(w)
obs = FixedIntervalNumberObserver(0.1, ('A', 'B', 'C'))
sim.run(10, obs)

There is a distinct difference in the second argument for **LatticeWorld**. This is called **Voxel radius**.
Spatiocyte defines the locations of the molecules with dividing the Space with molecule size, and call the minimum unit for this Space as **Voxel**.

In most cases the size of the molecule would be good for **Voxel radius**.

In this example, we set 5 $\mathrm{nm}$ as the radius of the molecule in the Space with the side 1 $\mathrm{\mu m}$ .

It takes more time to simulate, but the result is same with ODE or Gillespie.

## The diffusion movement of single molecule

## The diffusion coefficient and the second-order reaction

## The structure of Spatiocyte method

## The structure and the reaction